In [ ]:
!pip install transformers torch

In [ ]:
import os
import shutil
from pathlib import Path
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import mlflow
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository
from huggingface_model_wrapper import HuggingFaceModelWrapper


## Download and save model to a folder

In [ ]:

def save_model_to_dataset(model_name,dataset_name,project_id,mlflow_run_id):
    print(mlflow_run_id)
    artifact_dir = f"/mnt/data/{dataset_name}/mlflow-artifacts/{project_id}/{mlflow_run_id}"
    print(artifact_dir)
    Path(artifact_dir).mkdir(parents=True, exist_ok=True)
    # Load the model and tokenizer from Hugging Face
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print(artifact_dir)
    # Save the model and tokenizer to the local directory
    model.save_pretrained(artifact_dir)
    tokenizer.save_pretrained(artifact_dir)
    print(artifact_dir)
    return artifact_dir

## Create MLflow Client

In [ ]:
client = mlflow.MlflowClient()

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
# Define the folder where the model is saved
saved_model_path = "./saved_model"
mlflow_model_path = "./mlflow_huggingface_model"
project_id = os.environ['DOMINO_PROJECT_ID']
dataset_name = "LLMRegistrationToMLflow"

try:
    registered_model = client.create_registered_model(model_name)
except Exception as e:
    print(f"An error occurred: {e}")
    print("Can make it better, but verify that it RESOURCE_ALREADY_EXISTS")
    
# Log the Hugging Face model as an MLflow pyfunc model
experiment_name = 'HFModelv1'
exp = mlflow.get_experiment_by_name(experiment_name)
if not exp:
    mlflow.create_experiment(experiment_name)

In [ ]:
## Register the model

In [ ]:
mlflow.set_experiment(experiment_name)
with mlflow.start_run() as run:    
    run_id = run.info.run_id         
    artifact_dir = save_model_to_dataset(model_name,dataset_name,project_id,run_id)
    print(f"Model and tokenizer saved to {artifact_dir}")
    home_dir = os.path.expanduser("~")    
    local_path=f'{home_dir}/{model_name}'
    if os.path.exists(local_path):
        shutil.rmtree(local_path)
    model_info = mlflow.pyfunc.log_model(      
                    artifact_path = "",
                    python_model=HuggingFaceModelWrapper(),
                    artifacts={"model_path": artifact_dir}  # Pass the model path as metadata
                    )
    print(model_info)
    runs_uri = model_info.model_uri
    
    # Create a new model version of the RandomForestRegression model from this run
    desc = f"Version of the model {model_name}"
    model_src = RunsArtifactRepository.get_underlying_uri(runs_uri)
    mv = client.create_model_version(model_name, model_src, run.info.run_id, description=desc)
    print("Name: {}".format(mv.name))
    print("Version: {}".format(mv.version))
    print("Description: {}".format(mv.description))
    print("Status: {}".format(mv.status))
    print("Stage: {}".format(mv.current_stage))

In [ ]:
## Fetch the Registered Model By Name and Version and invoke it

In [ ]:
# Load the model from the registered model version
model_version = 6
model_uri = f"models:/{model_name}/{model_version}"
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Use the loaded model for inference
# Test input for inference
test_input = ["I love this movie!", "The cast could have been better!"]

# Invoke the model by passing input data
predictions = loaded_model.predict(test_input)

# Print the predictions
print(f"Predicted classes: {predictions}")

predictions = loaded_model.predict(test_input)

print(f"Predictions: {predictions}")

In [ ]:
import model_invocation_job 
model_invocation_job.invoke_model(model_name,model_version,"I loved the movie")